In [3]:
import os

#import open3d
from plyfile import PlyData, PlyElement
import numpy as np
import pandas as pd
from glob import glob
import torch
torch.cuda.is_available()

True

In [4]:
txt_path = './data/scannet/train/*'
txt_files = [file for file in glob(txt_path) if file.endswith('.txt')]
txt_files[:5]

['./data/scannet/train/scene0130_00.txt',
 './data/scannet/train/scene0072_01.txt',
 './data/scannet/train/scene0031_02.txt',
 './data/scannet/train/scene0174_00.txt',
 './data/scannet/train/scene0453_01.txt']

In [30]:
class_list = []
for filename in txt_files:
    with open(filename, 'r') as f:
        lines = f.read().split()
        class_list += [int(c) for c in lines]

In [68]:
len(class_list)

154073609

In [37]:
class_ids = (
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 84, 86, 87, 88, 89, 90, 93, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 110, 112, 115, 116, 118, 120, 121, 122, 125, 128, 130, 131, 132, 134, 136, 138, 139, 140, 141, 145, 148, 154,
155, 156, 157, 159, 161, 163, 165, 166, 168, 169, 170, 177, 180, 185, 188, 191, 193, 195, 202, 208, 213, 214, 221, 229, 230, 232, 233, 242, 250, 261, 264, 276, 283, 286, 300, 304, 312, 323, 325, 331, 342, 356, 370, 392, 395, 399, 408, 417,
488, 540, 562, 570, 572, 581, 609, 748, 776, 1156, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188, 1189, 1190, 1191)
label_dict = dict(zip(class_ids, np.arange(0, 201)))
label_dict

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 21: 19,
 22: 20,
 23: 21,
 24: 22,
 26: 23,
 27: 24,
 28: 25,
 29: 26,
 31: 27,
 32: 28,
 33: 29,
 34: 30,
 35: 31,
 36: 32,
 38: 33,
 39: 34,
 40: 35,
 41: 36,
 42: 37,
 44: 38,
 45: 39,
 46: 40,
 47: 41,
 48: 42,
 49: 43,
 50: 44,
 51: 45,
 52: 46,
 54: 47,
 55: 48,
 56: 49,
 57: 50,
 58: 51,
 59: 52,
 62: 53,
 63: 54,
 64: 55,
 65: 56,
 66: 57,
 67: 58,
 68: 59,
 69: 60,
 70: 61,
 71: 62,
 72: 63,
 73: 64,
 74: 65,
 75: 66,
 76: 67,
 77: 68,
 78: 69,
 79: 70,
 80: 71,
 82: 72,
 84: 73,
 86: 74,
 87: 75,
 88: 76,
 89: 77,
 90: 78,
 93: 79,
 95: 80,
 96: 81,
 97: 82,
 98: 83,
 99: 84,
 100: 85,
 101: 86,
 102: 87,
 103: 88,
 104: 89,
 105: 90,
 106: 91,
 107: 92,
 110: 93,
 112: 94,
 115: 95,
 116: 96,
 118: 97,
 120: 98,
 121: 99,
 122: 100,
 125: 101,
 128: 102,
 130: 103,
 131: 104,
 132: 105,
 134: 106,
 136: 107,
 138: 108,
 139: 

In [ ]:
uniq, cnt = np.unique(np.array(class_list), return_counts=True)

In [40]:
ids_cnt = {}
for i, ids in enumerate(uniq):
    if ids not in class_ids:
        ids = 0
        ids_cnt[ids] = ids_cnt[ids] + cnt[i]
    else:    
        ids_cnt[ids] = cnt[i]
ids_cnt

{0: 17883739,
 1: 36313663,
 2: 7123598,
 3: 28529463,
 4: 3659820,
 5: 4632889,
 6: 2913093,
 7: 2920352,
 8: 3132358,
 9: 2366714,
 10: 1162453,
 11: 2979623,
 13: 442810,
 14: 305419,
 15: 566288,
 16: 4725019,
 17: 354072,
 18: 2921535,
 19: 659321,
 21: 2244133,
 22: 1043172,
 23: 914823,
 24: 638861,
 26: 447682,
 27: 567653,
 28: 263972,
 29: 1426188,
 31: 237305,
 32: 508615,
 33: 419290,
 34: 356273,
 35: 238958,
 36: 522264,
 38: 238166,
 39: 37065,
 40: 765846,
 41: 3486721,
 42: 354233,
 44: 121994,
 45: 93102,
 46: 43669,
 47: 115181,
 48: 303151,
 49: 35727,
 50: 136587,
 51: 238518,
 52: 1069570,
 54: 146545,
 55: 336096,
 56: 982496,
 57: 375577,
 58: 369979,
 59: 125640,
 62: 276755,
 63: 65455,
 64: 132514,
 65: 32923,
 66: 18645,
 67: 144342,
 68: 139799,
 69: 203277,
 70: 97915,
 71: 262017,
 72: 302168,
 73: 25136,
 74: 302779,
 75: 244947,
 76: 82405,
 77: 42878,
 78: 337179,
 79: 27367,
 80: 79561,
 82: 85634,
 84: 28858,
 86: 170005,
 87: 48275,
 88: 4982,
 89: 

In [45]:
np.array(list(ids_cnt.values())).sum()

154073609

In [61]:
np.log(ids_cnt[0])

16.699402422178967

In [62]:
[np.log(cnt) for cnt in ids_cnt.values()]

[16.699402422178967,
 17.40770461960226,
 15.778923492827028,
 17.166447900720218,
 15.112924523840288,
 15.348691205553605,
 14.884725961281593,
 14.887214714924351,
 14.957296633519762,
 14.677013053143437,
 13.966042985518802,
 14.9073073404114,
 13.000896063120782,
 12.629439883321995,
 13.246858061695498,
 15.368382140065917,
 12.777255561265513,
 14.887619721052053,
 13.398965796558333,
 14.623829813030587,
 13.857776629303515,
 13.72648588292227,
 13.367442182310617,
 13.011838437926043,
 13.249265595525424,
 12.483598315897568,
 14.170515708573758,
 12.37710151257693,
 13.139446624260382,
 12.946318083601858,
 12.783452569884508,
 12.38404308325336,
 13.165928486144239,
 12.380723188519779,
 10.52042840709082,
 13.548736384124174,
 15.06447231098552,
 12.777710167692893,
 11.711727142178132,
 11.441450945311539,
 10.684393747202078,
 11.654260083085832,
 12.621986310191451,
 10.483661984436106,
 11.824717053215213,
 12.382200058160802,
 13.88276725650816,
 11.895087827508558,
 

In [88]:
ids_prob = {}
denom = np.sum([np.log(1/cnt) for cnt in ids_cnt.values()])
for keys, values in ids_cnt.items():
    #ids_prob[label_dict[keys]] = round(1 / (values/len(class_list)), 3)
    ids_prob[label_dict[keys]] =  round(np.log(1 / (values/len(class_list))), 3)
    
ids_prob

{0: 2.154,
 1: 1.445,
 2: 3.074,
 3: 1.686,
 4: 3.74,
 5: 3.504,
 6: 3.968,
 7: 3.966,
 8: 3.896,
 9: 4.176,
 10: 4.887,
 11: 3.946,
 12: 5.852,
 13: 6.224,
 14: 5.606,
 15: 3.485,
 16: 6.076,
 17: 3.965,
 18: 5.454,
 19: 4.229,
 20: 4.995,
 21: 5.126,
 22: 5.485,
 23: 5.841,
 24: 5.604,
 25: 6.369,
 26: 4.682,
 27: 6.476,
 28: 5.713,
 29: 5.907,
 30: 6.069,
 31: 6.469,
 32: 5.687,
 33: 6.472,
 34: 8.333,
 35: 5.304,
 36: 3.788,
 37: 6.075,
 38: 7.141,
 39: 7.411,
 40: 8.169,
 41: 7.199,
 42: 6.231,
 43: 8.369,
 44: 7.028,
 45: 6.471,
 46: 4.97,
 47: 6.958,
 48: 6.128,
 49: 5.055,
 50: 6.017,
 51: 6.032,
 52: 7.112,
 53: 6.322,
 54: 7.764,
 55: 7.058,
 56: 8.451,
 57: 9.02,
 58: 6.973,
 59: 7.005,
 60: 6.631,
 61: 7.361,
 62: 6.377,
 63: 6.234,
 64: 8.721,
 65: 6.232,
 66: 6.444,
 67: 7.534,
 68: 8.187,
 69: 6.125,
 70: 8.636,
 71: 7.569,
 72: 7.495,
 73: 8.583,
 74: 6.809,
 75: 8.068,
 76: 10.339,
 77: 6.122,
 78: 7.79,
 79: 7.174,
 80: 8.761,
 81: 5.647,
 82: 6.633,
 83: 8.729,
 84: 

In [109]:
np.mean(a), np.median(a)

(7.769741293532338, 8.004)

In [116]:
a = np.array(list(ids_prob.values()))
a[:10]

array([2.154, 1.445, 3.074, 1.686, 3.74 , 3.504, 3.968, 3.966, 3.896,
       4.176])

In [106]:
a = list(ids_prob.values())
np.round(a / np.median(a), 3)

array([0.269, 0.181, 0.384, 0.211, 0.467, 0.438, 0.496, 0.496, 0.487,
       0.522, 0.611, 0.493, 0.731, 0.778, 0.7  , 0.435, 0.759, 0.495,
       0.681, 0.528, 0.624, 0.64 , 0.685, 0.73 , 0.7  , 0.796, 0.585,
       0.809, 0.714, 0.738, 0.758, 0.808, 0.711, 0.809, 1.041, 0.663,
       0.473, 0.759, 0.892, 0.926, 1.021, 0.899, 0.778, 1.046, 0.878,
       0.808, 0.621, 0.869, 0.766, 0.632, 0.752, 0.754, 0.889, 0.79 ,
       0.97 , 0.882, 1.056, 1.127, 0.871, 0.875, 0.828, 0.92 , 0.797,
       0.779, 1.09 , 0.779, 0.805, 0.941, 1.023, 0.765, 1.079, 0.946,
       0.936, 1.072, 0.851, 1.008, 1.292, 0.765, 0.973, 0.896, 1.095,
       0.706, 0.829, 1.091, 0.781, 1.14 , 1.081, 1.121, 1.198, 1.044,
       1.003, 0.998, 1.029, 0.975, 1.124, 1.315, 1.016, 1.238, 0.943,
       1.072, 1.   , 0.739, 0.71 , 1.106, 0.929, 1.038, 1.028, 0.991,
       1.211, 1.073, 1.072, 0.913, 1.137, 1.345, 0.925, 1.237, 0.972,
       1.314, 0.769, 0.653, 1.123, 0.889, 1.218, 1.116, 1.297, 1.245,
       1.042, 1.155,

In [108]:
b = list(ids_prob.values())
np.round(b / np.mean(b), 3)

array([0.277, 0.186, 0.396, 0.217, 0.481, 0.451, 0.511, 0.51 , 0.501,
       0.537, 0.629, 0.508, 0.753, 0.801, 0.722, 0.449, 0.782, 0.51 ,
       0.702, 0.544, 0.643, 0.66 , 0.706, 0.752, 0.721, 0.82 , 0.603,
       0.833, 0.735, 0.76 , 0.781, 0.833, 0.732, 0.833, 1.072, 0.683,
       0.488, 0.782, 0.919, 0.954, 1.051, 0.927, 0.802, 1.077, 0.905,
       0.833, 0.64 , 0.896, 0.789, 0.651, 0.774, 0.776, 0.915, 0.814,
       0.999, 0.908, 1.088, 1.161, 0.897, 0.902, 0.853, 0.947, 0.821,
       0.802, 1.122, 0.802, 0.829, 0.97 , 1.054, 0.788, 1.111, 0.974,
       0.965, 1.105, 0.876, 1.038, 1.331, 0.788, 1.003, 0.923, 1.128,
       0.727, 0.854, 1.123, 0.805, 1.174, 1.114, 1.155, 1.234, 1.075,
       1.033, 1.028, 1.06 , 1.005, 1.158, 1.355, 1.047, 1.275, 0.972,
       1.104, 1.03 , 0.761, 0.732, 1.139, 0.957, 1.069, 1.059, 1.021,
       1.248, 1.105, 1.104, 0.94 , 1.172, 1.386, 0.953, 1.274, 1.002,
       1.354, 0.793, 0.672, 1.157, 0.916, 1.255, 1.15 , 1.336, 1.283,
       1.074, 1.189,

In [7]:
def read_plyfile(filepath):
    """
    Read ply file and return it as np.array. Returns None if empty
    """
    with open(filepath, 'rb') as f:
        plydata = PlyData.read(f)
    if plydata.elements:
        return pd.DataFrame(plydata.elements[0].data)#.values

In [8]:
train_root = './data/scannet'
path = os.path.join(train_root, train_files[0])
data = read_plyfile(path)
data

,x,y,z,nx,ny,nz,red,green,blue,label
0,-3.410296,1.226048,0.198699,0.0,0.0,0.0,101,107,90,19
1,-3.397429,1.258440,0.197952,0.0,0.0,0.0,88,83,78,19
2,-3.410380,1.187847,0.109613,0.0,0.0,0.0,39,39,35,19
3,-3.412473,1.221096,0.113794,0.0,0.0,0.0,132,117,108,19
4,-3.419209,1.238158,0.093539,0.0,0.0,0.0,101,88,76,0
...,...,...,...,...,...,...,...,...,...,...
81364,3.332217,-0.921485,2.638611,0.0,0.0,0.0,176,157,124,1
81365,3.325372,-0.915128,2.601836,0.0,0.0,0.0,177,161,128,1
81366,3.327703,-0.704435,2.513598,0.0,0.0,0.0,242,212,144,1
81367,3.332743,-0.670377,2.546573,0.0,0.0,0.0,175,161,127,1
